In [18]:
import pandas as pd
import xarray as xr

from risk_data import get_factor_master, get_yahoo_data_set, get_portfolios
from risk_stats import calculate_returns_set, align_dates
from risk_config import FACTOR_SET
from risk_util import safe_reindex

factor_set = FACTOR_SET
factor_set = 'read_short'

In [19]:
factor_master = get_factor_master(file_name='factor_master.xlsx', sheet_name=factor_set)
factor_list = factor_master.index
diffusion_map = factor_master['diffusion_type']
multiplier_map = factor_master['multiplier']


In [20]:
factor_list_yf = factor_master.query('source=="yfinance"').index
levels_yf = (get_yahoo_data_set(asset_names = factor_list_yf.tolist(), 
                                tickers = factor_master.loc[factor_list, 'ticker'],
                                field_name = 'Adj Close')
                .pipe(align_dates, ['SPY'])
                )


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [21]:
ret_yf = calculate_returns_set(levels_yf, 
                                periods=1,
                                diffusion_map=diffusion_map, 
                                multiplier_map=multiplier_map)
ret_list = [ret_yf]


In [22]:
ret_yf

factor_name,SPY,IWM,MDY,RSP,QQQ
date,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN
1993-02-01,71.123890,NaN,NaN,NaN,NaN
1993-02-02,21.185241,NaN,NaN,NaN,NaN
1993-02-03,105.709229,NaN,NaN,NaN,NaN
1993-02-04,41.843225,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-05-02,148.441984,224.919072,235.267310,176.425058,148.438673
2025-05-05,-57.343495,-76.815309,-19.029787,-33.160281,-59.325205
2025-05-06,-83.583643,-109.078021,-72.715988,-74.130529,-93.017288


In [23]:
factor_list_composite = factor_master.query('source==composite').index
not factor_list_composite.empty

False

In [26]:

factor_list_composite = factor_master.query('source==composite').index
if not factor_list_composite.empty:
    portfolios_weights = (get_portfolios()
                        #   .loc[factor_list_composite]
                        .pipe(safe_reindex, factor_master)
                        .fillna(0)
                        .loc[factor_list_yf]
                        )
    portfolios_ret = ret_yf @ portfolios_weights
    ret_list.append(portfolios_ret)



In [27]:

levels_latest = levels_yf.iloc[-1]

factor_data = xr.Dataset()
factor_data['ret']  = pd.concat(ret_list, axis=1).rename_axis(columns='factor_name')

In [28]:
factor_data

<xarray.Dataset> Size: 390kB
Dimensions:      (date: 8125, factor_name: 5)
Coordinates:
  * date         (date) datetime64[ns] 65kB 1993-01-29 1993-02-01 ... 2025-05-08
  * factor_name  (factor_name) object 40B 'SPY' 'IWM' 'MDY' 'RSP' 'QQQ'
Data variables:
    ret          (date, factor_name) float64 325kB nan nan nan ... 157.2 173.8